In [1]:
##process of choosing optimal parameter once optimal algorithm is chosen is called hypertuning

In [2]:
from sklearn import svm,datasets
iris = datasets.load_iris()

In [6]:
import pandas as pd
df = pd.DataFrame(iris.data,columns=iris.feature_names)
df['flower'] = iris.target
df['flower'] = df['flower'].apply(lambda x:iris.target_names[x])
df[47:53]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
47,4.6,3.2,1.4,0.2,setosa
48,5.3,3.7,1.5,0.2,setosa
49,5.0,3.3,1.4,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor
52,6.9,3.1,4.9,1.5,versicolor


Using train_test_split as traditionally

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(iris.data,iris.target,test_size=0.3)

### Trying SVM Model

In [12]:
model = svm.SVC(kernel='rbf',C=30,gamma='auto')
model.fit(X_train,y_train)
model.score(X_test,y_test)


#this method cannot be relied as score changes depending on X_test and X_train


0.8888888888888888

### Using Cross Fold Validation to calculate score

In [13]:
from sklearn.model_selection import cross_val_score


Trying out on diff values of kernal and C

In [15]:
cross_val_score(svm.SVC(kernel='linear',C=10,gamma='auto'),iris.data,iris.target,cv=5)

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [16]:
cross_val_score(svm.SVC(kernel='rbf',C=10,gamma='auto'),iris.data,iris.target,cv=5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [17]:
cross_val_score(svm.SVC(kernel='rbf',C=20,gamma='auto'),iris.data,iris.target,cv=5)

array([0.96666667, 1.        , 0.9       , 0.96666667, 1.        ])

The arrays above are result scores of 5(cv=5) iterations

In [18]:
# using for loop to try diffrent combinations


In [19]:
import numpy as np
kernels = ['rbf','linear']
C= [1,10,20]
avg_scores = {}
for kval in kernels:
    for cval in C:
        cv_scores = cross_val_score(svm.SVC(kernel=kval,C=cval,gamma='auto'),iris.data,iris.target,cv=5)
        avg_scores[kval + '_' + str(cval)] = np.average(cv_scores)

avg_scores


{'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668,
 'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666}

#### Implementing above task simply using GridSearchCV 

In [21]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(svm.SVC(gamma='auto'),{
    'C':[1,10,20],
    'kernel' : ['rbf','kernel']
},cv=5,return_train_score=False)

In [22]:
clf.fit(iris.data,iris.target)



C:\Python39\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
15 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Python39\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Python39\lib\site-packages\sklearn\svm\_base.py", line 255, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "C:\Python39\lib\site-packages\sklearn\svm\_base.py", line 315, in _dense_fit
    ) = libsvm.fit(
  File "sklearn\svm\_libsvm.pyx", line 176, in sklearn.svm._libsvm.fit
ValueError: 'k

GridSearchCV(cv=5, estimator=SVC(gamma='auto'),
             param_grid={'C': [1, 10, 20], 'kernel': ['rbf', 'kernel']})

In [23]:
clf.cv_results_

{'mean_fit_time': array([0.00179415, 0.00039067, 0.00019908, 0.00020041, 0.00119781,
        0.00059867]),
 'std_fit_time': array([0.000747  , 0.00047941, 0.00039816, 0.00040083, 0.00040113,
        0.00048882]),
 'mean_score_time': array([0.00019898, 0.        , 0.00039883, 0.        , 0.00019894,
        0.        ]),
 'std_score_time': array([0.00039797, 0.        , 0.00048846, 0.        , 0.00039787,
        0.        ]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'kernel', 'rbf', 'kernel', 'rbf', 'kernel'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'kernel'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'kernel'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20, 'kernel': 'kernel'}],


In [26]:
df = pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001794,0.000747,0.000199,0.000398,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.000391,0.000479,0.000000,0.000000,1,kernel,"{'C': 1, 'kernel': 'kernel'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
2,0.000199,0.000398,0.000399,0.000488,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.000200,0.000401,0.000000,0.000000,10,kernel,"{'C': 10, 'kernel': 'kernel'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
4,0.001198,0.000401,0.000199,0.000398,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,3
5,0.000599,0.000489,0.000000,0.000000,20,kernel,"{'C': 20, 'kernel': 'kernel'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,6


In [27]:
clf.best_estimator_

SVC(C=1, gamma='auto')

In [28]:
clf.best_params_

{'C': 1, 'kernel': 'rbf'}

In [29]:
clf.best_score_

0.9800000000000001

So many permutations GridSearch will go so instead using RandomizedSearch

In [31]:
from sklearn.model_selection import RandomizedSearchCV

rs = RandomizedSearchCV(svm.SVC(gamma='auto'),{
    'C':[1,10,20],
    'kernel' : ['rbf','kernel']
},cv=5,return_train_score=False,
                        n_iter = 2   #will try only 2 combinations out of total combinations
                        )

rs.fit(iris.data,iris.target)

pd.DataFrame(rs.cv_results_)[['param_C','param_kernel','mean_test_score']]


C:\Python39\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Python39\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Python39\lib\site-packages\sklearn\svm\_base.py", line 255, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "C:\Python39\lib\site-packages\sklearn\svm\_base.py", line 315, in _dense_fit
    ) = libsvm.fit(
  File "sklearn\svm\_libsvm.pyx", line 176, in sklearn.svm._libsvm.fit
ValueError: 'ker

,param_C,param_kernel,mean_test_score
0,10,rbf,0.98
1,1,kernel,NaN


### Choosing Best Model

In [32]:
from sklearn import svm



In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [34]:
model_params = {
    'svm' :{
        'model': svm.SVC(gamma='auto'),
        'params': {
            'C':[1,10,20],
            'kernel' : ['rbf','linear']
        }
    },
    'random_forest' : {
        'model' : RandomForestClassifier(),
        'params' : {
            'n_estimators' :[1,5,10]
        }        
    },
    'logistic_regression':{
        'model' : LogisticRegression(solver='liblinear',multi_class='auto'),
        'params' : {
            'C':[1,5,10]
        }
    }
}

In [35]:
scores = []

for model_name,mp in model_params.items():
    clf = GridSearchCV(mp['model'],mp['params'],cv=5,return_train_score=False)
    clf.fit(iris.data,iris.target)
    scores.append({
        'model':model_name,
        'best_score':clf.best_score_,
        'best_params':clf.best_params_
    })
    

In [36]:
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.966667,{'n_estimators': 10}
2,logistic_regression,0.966667,{'C': 5}


### the above approach has given which model among these is best and also hypertuning